## Explore

Reference：
* http://jakeboxer.com/blog/2009/12/13/the-knuth-morris-pratt-algorithm-in-my-own-words/
* http://www.ruanyifeng.com/blog/2013/05/Knuth–Morris–Pratt_algorithm.html

In [1]:
import string
import random
import time

In [2]:
sample_pattern = 'abcdabd'

In [3]:
cur_prefix = []
cur_suffix = []
partial_match_value = [0]
for i in range(2,len(sample_pattern)+1):
    partial_pattern = sample_pattern[:i]
    head = partial_pattern[-2]
    tail = partial_pattern[-1]
    if len(cur_prefix) == 0:
        cur_prefix.append(head)
    else:
        p = cur_prefix[-1]
        new_p = p+head
        cur_prefix.append(new_p)
    
    new_suffix = []
    for s in cur_suffix:
        new_s = s+tail
        new_suffix.append(new_s)
    new_suffix.append(tail)
    cur_suffix = new_suffix
    
    max_overlap = 0
    for st in set.intersection(set(cur_suffix),set(cur_prefix)):
        if len(st)>max_overlap:
            max_overlap = len(st)
    partial_match_value.append(max_overlap)
    print(i,partial_pattern,max_overlap,cur_prefix,cur_suffix)
    print(partial_match_value)

(2, 'ab', 0, ['a'], ['b'])
[0, 0]
(3, 'abc', 0, ['a', 'ab'], ['bc', 'c'])
[0, 0, 0]
(4, 'abcd', 0, ['a', 'ab', 'abc'], ['bcd', 'cd', 'd'])
[0, 0, 0, 0]
(5, 'abcda', 1, ['a', 'ab', 'abc', 'abcd'], ['bcda', 'cda', 'da', 'a'])
[0, 0, 0, 0, 1]
(6, 'abcdab', 2, ['a', 'ab', 'abc', 'abcd', 'abcda'], ['bcdab', 'cdab', 'dab', 'ab', 'b'])
[0, 0, 0, 0, 1, 2]
(7, 'abcdabd', 0, ['a', 'ab', 'abc', 'abcd', 'abcda', 'abcdab'], ['bcdabd', 'cdabd', 'dabd', 'abd', 'bd', 'd'])
[0, 0, 0, 0, 1, 2, 0]


## Function Develop

Reference:

https://www.matrix67.com/blog/archives/115

In [30]:
# this table means if j+1 can not match, how many steps backward.
def partial_match_table_naive(pattern_st):
    cur_prefix = []
    cur_suffix = []
    partial_match_value = [0]
    for i in range(2,len(pattern_st)+1):
        partial_pattern = pattern_st[:i]
        head = partial_pattern[-2]
        tail = partial_pattern[-1]
        if len(cur_prefix) == 0:
            cur_prefix.append(head)
        else:
            p = cur_prefix[-1]
            new_p = p+head
            cur_prefix.append(new_p)

        new_suffix = []
        for s in cur_suffix:
            new_s = s+tail
            new_suffix.append(new_s)
        new_suffix.append(tail)
        cur_suffix = new_suffix

        max_overlap = 0
        for st in set.intersection(set(cur_suffix),set(cur_prefix)):
            if len(st)>max_overlap:
                max_overlap = len(st)
        partial_match_value.append(max_overlap)
    return partial_match_value

def kmp_naive(sentence, pattern_st):
    # return the position of the first match, if no, then -1
    p = partial_match_table_naive(pattern_st)
    j = 0
    for i in range(len(sentence)):
        while j > 0 and sentence[i] != pattern_st[j]:
            j = p[j-1]
        if sentence[i] == pattern_st[j]:
            j += 1
        if j == len(pattern_st):
            return i-j+1
    return -1
    

# this table means if j can not match, which position should j move to.
def partial_match_table_god(pattern_st):
    p = [0]
    j = 0
    for i in range(1,len(pattern_st)):
        while j > 0 and pattern_st[i] != pattern_st[j]:
            j = p[j-1]
        if pattern_st[i] == pattern_st[j]:
            j += 1
        p.append(j)
    return p

def kmp(sentence, pattern_st):
    # return the position of the first match, if no, then -1
    p = partial_match_table_god(pattern_st)
    j = 0
    for i in range(len(sentence)):
        while j > 0 and sentence[i] != pattern_st[j]:
            j = p[j-1]
        if sentence[i] == pattern_st[j]:
            j += 1
        if j == len(pattern_st):
            return i-j+1
    return -1

def naive_match(haystack, needle):
    n = len(haystack)
    m = len(needle)
    for i in range(n):
        j = i
        p = 0
        while j<n and p<m and haystack[j] == needle[p]:
            p += 1
            j += 1
        if p == m:
            return i
    return -1

In [5]:
partial_match_table_god('llla')

[0, 1, 2, 0]

In [6]:
kmp_naive('hello','ll')

2

In [7]:
kmp('aaaaa','bba')

-1

In [8]:
kmp('aaaabbba','bba')

5

In [9]:
naive_match('aaaabbba','bba')

5

## 性能测试

### 随机数据匹配

In [33]:
sts = []
pts = []
n_iter = 10
for iter in range(n_iter):
    sts.append(''.join([random.choice(string.ascii_letters + string.digits) for n in xrange(10000000)]))
    pts.append(''.join([random.choice(string.ascii_letters + string.digits) for n in xrange(2000)]))

In [34]:
start_time = time.time()
for i in range(n_iter):
    st = sts[i]
    pt = pts[i]
    naive_match(st,pt)
print('time used:',time.time()-start_time)

('time used:', 20.494627952575684)


In [35]:
start_time = time.time()
for iter in range(n_iter):
    st = sts[i]
    pt = pts[i]
    kmp(st,pt)
print('time used:',time.time()-start_time)

('time used:', 20.881771087646484)


结论：KMP 算法在随机数据匹配上，效果甚至不如朴素匹配算法，因为有额外的 partial_match_table 计算开销。

### 有关数据匹配

In [13]:
sts = []
pts = []
n_iter = 10000
rept = 1000
pt_len = 50
for iter in range(n_iter):
    pt = ''.join([random.choice(string.ascii_letters + string.digits) for n in xrange(pt_len)])
    pts.append(pt)
    st = ''
    for r in range(rept):
        rand_st = pt[:random.randint(1,pt_len)]
        st += rand_st
    sts.append(st)

In [14]:
start_time = time.time()
for i in range(n_iter):
    st = sts[i]
    pt = pts[i]
    naive_match(st,pt)
print('time used:',time.time()-start_time)

('time used:', 8.886002779006958)


In [15]:
start_time = time.time()
for iter in range(n_iter):
    st = sts[i]
    pt = pts[i]
    kmp(st,pt)
print('time used:',time.time()-start_time)

('time used:', 3.1221609115600586)


结论：KMP 算法在相关数据匹配上，效果要好于朴素匹配算法，此时 partial_match_table 发挥了作用。复杂度 O(m+n)

Reference:
* https://blog.csdn.net/zhanghaiyang9999/article/details/40709247

好了，我要去学 BM 算法了。